In [ ]:
import requests # pip install requests
import pandas # pip install pandas
import json


def get_with_credentials(tok, uri, **kwargs):
    # just in case we forget to put "Token MyToken"
    token = tok if tok.lower().startswith("token") else f"Token {tok}"
    headers = {"Authorization": token}
    return requests.get(uri, headers=headers, **kwargs)


def get_readings_response(sn, start_date, end_date, **extra_kwargs_for_endpoint):
    server = extra_kwargs_for_endpoint.get("server", "https://zentracloud.com")
    url = f"{server}/api/v4/get_readings/"

    default_args = {
        'output_format': "df",
        'per_page': 100000,
        'page_num': 1,
        'sort_by': 'asc',
        'start_date': start_date,
        'end_date': end_date
    }
    data = {**default_args, **extra_kwargs_for_endpoint, "device_sn": sn}
    tok = extra_kwargs_for_endpoint.pop("token", "")
    return get_with_credentials(tok, url, params=data)

def get_readings_dataframe(sn,start_date,end_date,**extra_kwargs_for_endpoint):
    res = get_readings_response(sn,start_date,end_date,**extra_kwargs_for_endpoint)
    if(res.ok):
        data = res.json()
        return pandas.DataFrame(**json.loads(data["data"]))
    return res

#fill in your token, device serial number and start and end dates here.
tok = ""
sn=""
start_date="2024-10-03 00:00:00"
end_date="2024-10-16 00:00:00"

#specify the server here.
server="https://zentracloud.com"
df = get_readings_dataframe(sn, start_date, end_date, token=tok, server=server)
print(df)

In [ ]:
pivoted_df = df.pivot_table(
    index=['timestamp_utc', 'datetime'],
    columns='measurement',
    values='value',
    aggfunc='first'  # Use 'first' to get the first non-null value
).reset_index()

# Optional: Flatten the multi-level column index
pivoted_df.columns.name = None  # Remove the column index name
pivoted_df.columns = [col if col is not None else '' for col in pivoted_df.columns]  # Handle None columns

In [ ]:
pivoted_df = pivoted_df.drop(['X-axis Level', 'Y-axis Level'], axis=1)
pivoted_df.head()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
    

In [ ]:
df = pivoted_df.copy()
sequence_length = 96
features = df.drop(['timestamp_utc', 'datetime' ], axis=1).values
df = df.ffill()

# Create sequences for prediction
X_test = []
for i in range(len(df) - sequence_length):
    X_test.append(features[i:i+sequence_length])
X_test = np.array(X_test)

# Normalize the features using StandardScaler
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [ ]:
# Model Loading
from tensorflow.keras.models import load_model
model = load_model('ResNet.keras')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Make predictions
predictions = model.predict(X_test)

# Get the class with highest probability for each prediction
predicted_classes = np.argmax(predictions, axis=1)
confidence_scores = np.max(predictions, axis=1)

# Create reverse mapping for class interpretation
class_mapping = {0: 'clean', 1: 'random', 2: 'malfunction', 3: 'drift', 4: 'bias'}

# Define confidence thresholds for each class
confidence_thresholds = {
    'clean': 0.9,
    'random': 0.9,
    'malfunction': 0.95,
    'drift': 0.95,
    'bias': 0.95
}

# Convert numeric predictions to class labels with confidence thresholds
predicted_labels = []
for pred, conf in zip(predicted_classes, confidence_scores):
    predicted_class = class_mapping[pred]
    # Check if confidence meets the threshold for the predicted class
    if conf >= confidence_thresholds[predicted_class]:
        predicted_labels.append(predicted_class)
    else:
        predicted_labels.append('clean')  # Default to clean if confidence is too low

# Print predictions and their confidence scores
print("\nSample Predictions:")
for label, conf in zip(predicted_labels[:50], confidence_scores[:50]):
    print(f"Class: {label}, Confidence: {conf:.4f}")

# FST Prediction

In [ ]:
# Read the data
df = pd.read_csv('FST_analysis.csv')

# Print columns for verification
print("Available columns:", df.columns.tolist())

# Combine Date and Time
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df = df.sort_values('DateTime')

# Add time-based features
df['Hour'] = df['DateTime'].dt.hour
df['DayOfWeek'] = df['DateTime'].dt.dayofweek
df['Month'] = df['DateTime'].dt.month

# Define features and target
features = ['Air Temperature', 'Dew Point', 'Solar Radiation', 'Wind Speed', 
           'Hour', 'DayOfWeek', 'Month']
target = 'FST_EB'

# Handle missing values and outliers
df[features] = df[features].ffill().bfill()

scaler_X = RobustScaler()
scaler_y = RobustScaler()

X = scaler_X.fit_transform(df[features])
y = scaler_y.fit_transform(df[[target]])

# Create sequences with overlap
sequence_length = 24  
stride = 1  
X_sequences = []
y_sequences = []

for i in range(0, len(df) - sequence_length, stride):
    X_sequences.append(X[i:(i + sequence_length)])
    y_sequences.append(y[i + sequence_length])

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

)


In [ ]:
# Model Loading
from tensorflow.keras.models import load_model
model = load_model('fst_model.keras')
model.compile(optimizer='adam', loss='huber')

In [ ]:
predictions = model.predict(X_sequences)
predictions = scaler_y.inverse_transform(predictions)

# Print some sample predictions
print("\nSample Predictions (FST_EB values):")
print(predictions[:5])